In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

load_dotenv()
api_key_1 = os.getenv('API_KEY') #Gjør dette med enn .env fil
print(api_key_1)
base_URL = os.getenv('DATABASE_URL')



if not api_key_1 or not base_URL:
    raise ValueError ("DATABASE/APIKEY mangler i .env filen")

city = "Trondheim"
start_date = datetime(2025, 1, 16)
end_date = datetime(2025, 3, 16)

vaerdata = []

current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    url = f"{base_URL}/history.json?key={api_key_1}&q={city}&dt={date_str}"



    response = requests.get(url)

    if response.status_code == 200: #Kontroll for vellykket API-kall
        data = response.json()
    else: 
        print(f"Feil ved henting av data fra API-et for {date_str} : {response.status_code} ")
        current_date += timedelta(days=1)
        continue


    for hour_data in data.get("forecast", {}).get("forecastday", [])[0].get("hour", []):
        tidspunkt = hour_data.get("time", "Ukjent tidspunk")
        temperatur = hour_data.get("temp_c", "Ukjent temperatur")
        vindhastighet = hour_data.get("wind_kph", "Ukjent vindhastighet")
        pressure_mb = hour_data.get("pressure_mb", "Ukjent trykk" )
        nedbør = hour_data.get("precip_mm", "Ukjent nedbør")

        vaerdata.append({
            "Tidspunkt" : tidspunkt, 
            "Temperatur (°C)" : temperatur, 
            "Vindhastighet (km/t)" : vindhastighet,
            "Trykk" : pressure_mb, 
            "Nedbør (mm)" : nedbør
        })

    current_date += timedelta (days=1)

df = pd.DataFrame(vaerdata)

df.to_csv("trondheim_vaerdata_full_1.csv", index = False, encoding="utf-8")

print("data lagret i trondheim_vaerdata_full_1.csv")

print(df.head())




ModuleNotFoundError: No module named 'requests'

In [ ]:
a